# SyftBox Notifications - End-to-End Test

This notebook demonstrates the complete notification system flow.

## Architecture Overview

### How Job Detection Works

The notification system uses **direct Google Drive polling** to detect new jobs without requiring a full sync:

```
┌─────────────────────────────────────────────────────────────────┐
│                        GOOGLE DRIVE                              │
│                                                                  │
│  syft_outbox_inbox_{ds_email}_to_{do_email}/                    │
│      └── msgv2_{timestamp}_{uuid}.tar.gz   ← Job submissions    │
└─────────────────────────────────────────────────────────────────┘
                              │
                              │ JobMonitor polls Drive directly
                              ▼
┌─────────────────────────────────────────────────────────────────┐
│  _find_inbox_folders()                                          │
│  Query: "name contains 'syft_outbox_inbox' AND                  │
│          name contains '_to_{do_email}'"                        │
│                                                                  │
│  Returns ALL folders where ANYONE sent to DO                    │
│  (not filtered by peer relationship)                            │
└─────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────┐
│  _parse_job_from_message()                                      │
│  Downloads msgv2_*.tar.gz → Extracts job info                   │
│  (job_name, submitter_email)                                    │
└─────────────────────────────────────────────────────────────────┘
```

### Key Insight: DO Receives Jobs from ANY Sender

**Q: How does DO(x) know to look for jobs from DS(y) and not DS(z)?**

**A: It doesn't filter! DO receives notifications for jobs from ALL senders.**

The folder naming convention `syft_outbox_inbox_{sender}_to_{recipient}` means:
- DO's JobMonitor queries for `*_to_{do_email}` 
- This matches ALL inbox folders addressed to DO, regardless of sender
- The peer relationship grants access to private data, not filtering who can submit

### Hybrid Approach

The JobMonitor uses a hybrid approach:
1. **Drive polling** for NEW job submissions (lightweight, no full sync)
2. **Local filesystem** for status changes (approved, executed markers)

This enables running as a standalone daemon without the full client.

## Prerequisites

### 1. Create credentials directory

**Colab:**
1. Go to Google Drive
2. Create folder: `syft-creds`

**Local/VM:**
```bash
mkdir -p ~/.syft-creds
```

### 2. Get OAuth credentials
1. Go to [Google Cloud Console](https://console.cloud.google.com/)
2. Create project → Enable Gmail API
3. APIs & Services → Credentials → Create OAuth 2.0 Client ID (Desktop app)
4. Download as `credentials.json`
5. Place in `syft-creds/` folder

## API
```python
from syft_client.notifications import NotificationMonitor

# One-time setup
NotificationMonitor.setup()

# Usage
monitor = NotificationMonitor.from_client(client_do)
monitor.start()        # Start all (jobs + peers)
monitor.start("jobs")  # Only job notifications
monitor.start("peers") # Only peer notifications
monitor.stop()
```

## 0. Cleanup Utility 

Run this cell to reset state for fresh testing.

In [ ]:
# CLEANUP UTILITY - Resets peers for fresh demo, keeps notification state
import shutil
from pathlib import Path

def cleanup_for_testing(
    email_do: str = "test1@openmined.org",
    email_ds: str = "test2@openmined.org",
):
    """
    Reset for fresh demo testing.
    
    - Deletes local SyftBox folders (will re-sync from Drive)
    - Keeps notification state (prevents duplicate emails)
    - Jobs use unique names, so no need to reset job state
    
    For peers: manually delete 'syft_outbox_inbox*' folders in Google Drive
    """
    # Paths
    syftbox_do = Path.home() / f"SyftBox_{email_do}"
    syftbox_ds = Path.home() / f"SyftBox_{email_ds}"
    creds_dir = Path.home() / ".syft-creds"
    state_file = creds_dir / "notification_state.json"
    
    print("🧹 Cleanup for fresh demo...")
    
    # Delete local SyftBox folders
    for syftbox in [syftbox_do, syftbox_ds]:
        if syftbox.exists():
            shutil.rmtree(syftbox)
            print(f"  ✅ Deleted: {syftbox.name}")
    
    # Reset peer snapshot in state (so new peer additions trigger emails)
    if state_file.exists():
        import json
        with open(state_file, "r") as f:
            state = json.load(f)
        
        # Clear peer-related state only
        state["peer_snapshot"] = []
        keys_to_remove = [k for k in state.get("notified_jobs", {}) if k.startswith("peer_")]
        for k in keys_to_remove:
            del state["notified_jobs"][k]
        
        with open(state_file, "w") as f:
            json.dump(state, f, indent=2)
        print(f"  ✅ Reset peer state (kept job state)")
    
    print("\n⚠️  To test peer notifications, also delete in Google Drive:")
    print("   Search: syft_outbox_inbox")
    print("   Delete all matches in BOTH accounts + empty trash")

cleanup_for_testing()

## 1. Imports and Configuration

In [ ]:
import syft_client as sc
import os
import json
from pathlib import Path

# Configure emails
email_do = os.environ.get("SYFT_EMAIL_DO", "test1@openmined.org")
email_ds = os.environ.get("SYFT_EMAIL_DS", "test2@openmined.org")

# Token paths for Google Drive access
token_path_do = sc.CREDENTIALS_DIR / "token_do.json"
token_path_ds = sc.CREDENTIALS_DIR / "token_ds.json"

print(f"DO: {email_do}")
print(f"DS: {email_ds}")
print(f"DO token: {token_path_do} - {'exists' if token_path_do.exists() else 'MISSING'}")
print(f"DS token: {token_path_ds} - {'exists' if token_path_ds.exists() else 'MISSING'}")

## 2. Gmail Notification Setup (One-time)

This step:
1. Looks for `credentials.json` in `syft-creds/`
2. Runs OAuth flow (opens browser)
3. Saves `gmail_token.json` to same directory

In [ ]:
from syft_client.notifications import NotificationMonitor

# Check where credentials should be
creds_dir = NotificationMonitor.get_creds_dir()
print(f"Credentials directory: {creds_dir}")
print(f"  credentials.json: {'exists' if (creds_dir / 'credentials.json').exists() else 'MISSING'}")
print(f"  gmail_token.json: {'exists' if (creds_dir / 'gmail_token.json').exists() else 'MISSING'}")

In [ ]:
# Run setup (skips if token already exists)
NotificationMonitor.setup()

## 3. Login as DO and Start Monitor

In [ ]:
# Login as Data Owner
client_do = sc.login_do(
    email=email_do,
    token_path=token_path_do,
)
print(f"DO logged in: {client_do.email}")
print(f"DO SyftBox: {client_do.syftbox_folder}")

In [ ]:
# Start notification monitor (runs in background)
monitor = NotificationMonitor.from_client(client_do)
monitor.start()  # Start all (jobs + peers)

print("Monitor started in background (jobs + peers)")

### What's Happening Behind the Scenes

When the monitor starts, `JobMonitor` uses **direct Drive polling** (not `client.sync()`):

```python
# JobMonitor._check_all_entities() does:

# 1. Poll Drive for new jobs (lightweight - no full sync!)
if self._drive_service:
    self._poll_drive_for_new_jobs()

# 2. Check local filesystem for status changes
self._check_local_for_status_changes()
```

**Drive polling flow:**
1. `_find_inbox_folders()` - Queries Drive for `syft_outbox_inbox_*_to_{do_email}` folders
2. `_get_pending_messages()` - Lists `msgv2_*.tar.gz` files in each folder
3. `_parse_job_from_message()` - Downloads and parses message to extract job info
4. Sends notification if job is new

**Benefits:**
- Only reads inbox folders (not full sync)
- Can run as standalone daemon without full client
- Falls back to `client.sync()` if no Drive token available

## 4. Login as DS

In [ ]:
# Login as Data Scientist
client_ds = sc.login_ds(
    email=email_ds,
    token_path=token_path_ds,
)
print(f"DS logged in: {client_ds.email}")
print(f"DS SyftBox: {client_ds.syftbox_folder}")

## 5. DS Adds DO as Peer

PeerMonitor detects this and sends:
- Email to DO: "New peer request from DS"
- Email to DS: "Your request was sent to DO"

In [ ]:
# DS adds DO as peer
client_ds.add_peer(email_do)
print(f"DS added {email_do} as peer")
print("PeerMonitor will detect this and send notifications...")

## 6. DO Accepts Peer Request

When DO adds DS back, we manually notify DS that the request was accepted.

In [ ]:
# DO adds DS back (accepts peer request)
client_do.add_peer(email_ds)
print(f"DO added {email_ds} as peer")

# Manually notify DS that peer request was accepted
monitor.notify_peer_granted(email_ds)
print("Notification sent to DS about peer request acceptance")

## 7. Create Dataset (DO)

In [ ]:
# Download sample dataset
MOCK_DATASET_URL = "https://raw.githubusercontent.com/OpenMined/datasets/refs/heads/main/beach/sales-dataset/mock/sales.csv"
PRIVATE_DATASET_URL = "https://raw.githubusercontent.com/OpenMined/datasets/refs/heads/main/beach/sales-dataset/private/sales.csv"

!curl -s -o /tmp/sales_mock.csv $MOCK_DATASET_URL
!curl -s -o /tmp/sales_private.csv $PRIVATE_DATASET_URL
!echo "# Sales Dataset\nSample sales data for testing." > /tmp/readme.md

print("Dataset files downloaded")

In [ ]:
# Create dataset (clean up if already exists)
import shutil

# Clean up both public and private dataset directories
public_dataset_path = client_do.syftbox_folder / client_do.email / "public" / "syft_datasets" / "sales-data"
private_dataset_path = client_do.syftbox_folder / "private" / "syft_datasets" / "sales-data"

for path in [public_dataset_path, private_dataset_path]:
    if path.exists():
        print(f"Cleaning up {path}...")
        shutil.rmtree(path)

client_do.create_dataset(
    name="sales-data",
    mock_path="/tmp/sales_mock.csv",
    private_path="/tmp/sales_private.csv",
    summary="Sales data for analysis",
    readme_path="/tmp/readme.md",
    tags=["sales", "test"],
)
print("Dataset created")

# Sync
client_do.sync()
print("DO synced")

In [ ]:
# DS syncs to see dataset
client_ds.sync()
print("DS synced")
print("Available datasets:", client_ds.datasets.get_all())

## 8. DS Submits Job

When DS submits a job:
1. DS writes job files locally to `SyftBox/{do_email}/app_data/job/{job_name}/`
2. Files are packaged into `ProposedFileChangesMessage` (`.tar.gz`)
3. Message uploaded to Drive: `syft_outbox_inbox_{ds_email}_to_{do_email}/msgv2_*.tar.gz`

**JobMonitor detects this via direct Drive polling:**
1. Queries for folders matching `*_to_{do_email}` (finds all senders)
2. Lists `msgv2_*.tar.gz` messages in each folder
3. Downloads and parses to find job submissions
4. Sends notification to DO

**No `client.sync()` needed!** The monitor polls Drive directly.

JobMonitor will send:
- Email to DO: "New job from DS"

In [ ]:
# Create job code
job_code = '''
import os
import json
import syft_client as sc

# Access private data
data_path = "syft://private/syft_datasets/sales-data/sales_private.csv"
resolved_path = sc.resolve_path(data_path)

with open(resolved_path, "r") as f:
    data = f.read()

# Count lines
line_count = len(data.strip().split("\\n"))

# Save result
os.mkdir("outputs")
with open("outputs/result.json", "w") as f:
    json.dump({"line_count": line_count, "status": "success"}, f)
'''

with open("/tmp/analyze_sales.py", "w") as f:
    f.write(job_code)
    
print("Job code written")

In [ ]:
# Submit job with unique name to ensure fresh notifications
import shutil
import uuid

# Generate unique job name for this run
job_name = f"sales_analysis_{uuid.uuid4().hex[:8]}"

# The job is stored under the target user's directory in the DS syftbox
job_dir_in_ds = client_ds.syftbox_folder / email_do / "app_data" / "job" / job_name
job_dir_in_do = client_do.syftbox_folder / email_do / "app_data" / "job" / job_name

for job_dir in [job_dir_in_ds, job_dir_in_do]:
    if job_dir.exists():
        print(f"Cleaning up existing job: {job_dir}")
        shutil.rmtree(job_dir)

client_ds.submit_python_job(
    user=email_do,
    code_path="/tmp/analyze_sales.py",
    job_name=job_name,
)
print(f"Job '{job_name}' submitted to {email_do}")
print("JobMonitor will detect this and notify DO...")

In [ ]:
# JobMonitor auto-syncs every 10 seconds, so DO will receive the job automatically.
# This manual sync just speeds up the demo - you can skip it and wait for the monitor.
client_do.sync()
print(f"DO synced - Jobs received: {len(client_do.job_client.jobs)}")

## 9. DO Approves Job

JobMonitor detects this and sends:
- Email to DS: "Your job was approved"

In [ ]:
# Get and approve the job
job = client_do.job_client.jobs[0]
print(f"Job: {job.name}")
print(f"Submitted by: {job.submitted_by}")

job.approve()
print(f"Job '{job.name}' approved")
print("JobMonitor will detect this and notify DS...")

## 10. DO Executes Job

JobMonitor detects this and sends:
- Email to DS: "Your job completed"

In [ ]:
# Execute approved jobs
client_do.job_runner.process_approved_jobs()
print("Job executed")
print("JobMonitor will detect this and notify DS...")

# Sync results
client_do.sync()
print("DO synced results")

## 11. DS Receives Results

In [ ]:
# DS syncs to get results
client_ds.sync()
print("DS synced")

# Check results
ds_job = client_ds.job_client.jobs[-1]
print(f"Job: {ds_job.name}")
print(f"Output paths: {ds_job.output_paths}")

if ds_job.output_paths:
    with open(ds_job.output_paths[0], "r") as f:
        result = json.load(f)
    print(f"Result: {result}")

## 12. Stop Monitor

In [ ]:
# Stop the monitor
monitor.stop()
print("Monitor stopped")

## 13. Check Notification State

In [ ]:
# View what notifications were sent
state_path = NotificationMonitor.get_creds_dir() / "notification_state.json"

if state_path.exists():
    with open(state_path) as f:
        state_data = json.load(f)
    print("Notifications sent:")
    print(json.dumps(state_data, indent=2))
else:
    print("No state file found")

## Summary

Check your inboxes for:

**To DO (test1@openmined.org):**
- New Peer Request from DS
- New Job: analyze_sales_job

**To DS (test2@openmined.org):**
- Peer Request Sent confirmation
- Peer Request Accepted by DO
- Job Approved: analyze_sales_job
- Job Completed: analyze_sales_job